In [1]:
import pandas as pd
import numpy as np
from IPython.display import HTML
#import langdetect

from langdetect import detect, detect_langs

In [37]:
def combineCSVs():
  dft = pd.read_csv('https://raw.githubusercontent.com/oskarkris/cluster_project/main/csvdata/savedrecs%20(1).csv')
  print("read first")
  for n in range(2,13):
    print("read ", n)
    df_temp = pd.read_csv("https://raw.githubusercontent.com/oskarkris/cluster_project/main/csvdata/savedrecs%20("+str(n)+").csv")
    dft = pd.concat([dft, df_temp])
  return dft

def isEmpty(abstract):
    if abstract==" " or abstract=="":
        return True
    else:
        return False

def isEnglish(abstract):
    try:
        detected = detect_langs(abstract)
        lang = detected[0].lang
        prob = detected[0].prob
        if lang == "en" and prob > 0.95:
            return True
        else:
            return False
    except:
        return False

def make_clickable(url, name):
    return '<a href="{}" rel="noopener noreferrer" target="_blank">{}</a>'.format(url, name)


def remove_decimal(value):
    return str(value).replace('.0', '')

def generateDF():
    dft = combineCSVs()
    dft = dft.filter(["Publication Year", "Abstract", "UT (Unique WOS ID)", "Cited Reference Count", "Article Title"])
    dft = dft.rename(columns={"Publication Year" : "year", "UT (Unique WOS ID)": "wos", "Article Title": "title",  "Cited Reference Count": "citations", "Abstract": "abstract"})
    dft['year'] = dft['year'].apply(remove_decimal)
    dft["index"] = range(0, len(dft.index))
    dft["en"] = dft.apply(lambda x: isEnglish(x["abstract"]), axis=1)
    #Adds a link to Web of Science
    dft["link"] = dft.apply(lambda x: make_clickable("https://www.webofscience.com/wos/woscc/full-record/" + x["wos"], "link"), axis=1)
    dft = dft[['index', 'title', 'year', 'wos', 'en', 'link', 'abstract', 'citations']]
    return dft


def render(df):
    return HTML(df.to_html(render_links=True, escape=False))


In [40]:

create_new = True
if create_new:
    df = generateDF()
    df.head()
    df.loc[df['en'] == True].to_csv(r'master.csv', index=False)
    df.loc[df['en'] == False].to_csv(r'missingAbstracts.csv', index=False)
df = pd.read_csv(r'master.csv')
dm = pd.read_csv(r'missingAbstracts.csv')



read first
read  2
read  3
read  4
read  5
read  6
read  7
read  8
read  9
read  10
read  11
read  12


In [41]:
df.head()


,index,title,year,wos,en,link,abstract,citations
0,0,Financing Climate Change Adaptation: Internati...,2021.0,WOS:000666451800001,True,"<a href=""https://www.webofscience.com/wos/wosc...",Climate change adaptation is one of the main s...,27
1,1,Driving the agenda for climate change adaptati...,2013.0,WOS:000320956800008,True,"<a href=""https://www.webofscience.com/wos/wosc...",The absence of clear signals from the Norwegia...,55
2,2,Climate change adaptation policies and plans: ...,2018.0,WOS:000417078200110,True,"<a href=""https://www.webofscience.com/wos/wosc...",Building a climate-ready adaptation society is...,48
3,3,Place-based climate change adaptation: A criti...,2014.0,WOS:000345061100013,True,"<a href=""https://www.webofscience.com/wos/wosc...",Climate change adaptation frameworks often emp...,66
4,4,Responsibility for private sector adaptation t...,2014.0,WOS:000338711600014,True,"<a href=""https://www.webofscience.com/wos/wosc...",The Intergovernmental Panel on Climate Change ...,56


In [ ]:
from IPython.display import clear_output

def addAbstractsManually(missingAbstracts):
    indices = (pd.read_csv(r"manualAbstracts.csv")["index"]).values.tolist()
    abstract = pd.read_csv(r"manualAbstracts.csv")['abstract'].values.tolist()
    missingAbstracts.set_index('index', drop=False, inplace=True)
    for index, row in missingAbstracts.iterrows(): 
        if index not in indices:
            print("If the")
            print("index: ", index, "\n", flush=True)
            print("Abstract: ", "\n", row["abstract"], flush=True)
            print("Link: ", "\n", row["link"], flush=True)
            a = input("Insert new abstract. Leave blank if you find no abstract")
            indices.append(index)
            abstract.append(a)
            
        else:
            print("item already has a manually inserted abstract")
        clear_output(wait=True)

    data = {'index': indices,
                'abstract': abstract}
    pd.DataFrame(data).to_csv(r"manualAbstracts.csv", index=False)



In [ ]:
# NB
# Du blir prompta til å oppgi abstrakt til en artikkel som mangler det når du kjører funksjonen
# Svaret ditt lagres i "manualAbstracts.csv"
# Om det ikke finnes et abstract sett det til "" - altså ikke legg til informasjon, bare press ENTER
# Når du er ferdig trykk ESCAPE
man = False
if man:
    addAbstractsManually(dm)

If the
index:  7790 

Abstract:  
 nan
Link:  
 <a href="https://www.webofscience.com/wos/woscc/full-record/WOS:000377053000007" rel="noopener noreferrer" target="_blank">link</a>


KeyboardInterrupt: Interrupted by user